In [1]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors

import pandas as pd
import xarray as xr
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime, timedelta
import os

import seaborn as sns
import cmocean as cmocn

%matplotlib inline

In [2]:
ds= xr.open_dataset('/srv/data2/srai_poseidon/srai_poseidon/observation/SatelliteVsBuoy/downloads/Buoy/extractedGZ/WINDS/T_09N_140W_xrr_MatchUp_720_mins_2000.nc')
ds

<xarray.Dataset> Size: 139MB
Dimensions:                           (QS_TIME: 6044, cross_track: 3,
                                       ambiguities: 4, TIME: 6762,
                                       TAO_TIME_INDEX: 73, HEIGHT: 3, DEPTH: 1)
Coordinates:
  * QS_TIME                           (QS_TIME) datetime64[ns] 48kB 2000-01-0...
  * cross_track                       (cross_track) int64 24B 0 1 2
  * ambiguities                       (ambiguities) int64 32B 0 1 2 3
  * TIME                              (TIME) datetime64[ns] 54kB 2000-01-01T0...
  * TAO_TIME_INDEX                    (TAO_TIME_INDEX) int64 584B -36 -35 ... 36
  * HEIGHT                            (HEIGHT) float32 12B 3.0 4.0 10.0
  * DEPTH                             (DEPTH) float32 4B 1.0
Data variables: (12/1035)
    retrieved_wind_speed              (QS_TIME, cross_track) float64 145kB ...
    retrieved_wind_direction          (QS_TIME, cross_track) float64 145kB ...
    rain_impact                       (QS_TIME, cross_track) float64 145kB ...
    flags                             (QS_TIME, cross_track) float64 145kB ...
    eflags                            (QS_TIME, cross_track) float64 145kB ...
    nudge_wind_speed                  (QS_TIME, cross_track) float64 145kB ...
    ...                                ...
    mean WDIR 720min                  (QS_TIME) float64 48kB ...
    std. dev. WDIR 720min             (QS_TIME) float64 48kB ...
    mean cosWDIR 720min               (QS_TIME) float64 48kB ...
    std. dev. cosWDIR 720min          (QS_TIME) float64 48kB ...
    mean sinWDIR 720min               (QS_TIME) float64 48kB ...
    std. dev. sinWDIR 720min          (QS_TIME) float64 48kB ...

In [10]:
print(list(np.unique(ds.WSPD_QC.to_numpy())))
print(list(np.unique(ds.WDIR_QC.to_numpy())))
print(list(np.unique(ds.SST_QC.to_numpy())))
print(list(np.unique(ds.RELH_QC.to_numpy())))
print(list(np.unique(ds.AIRT_QC.to_numpy())))
                                                                                                                                                                                              

[np.float32(2.0), np.float32(4.0), np.float32(9.0), np.float32(nan)]
[np.float32(2.0), np.float32(4.0), np.float32(9.0), np.float32(nan)]
[np.float32(2.0), np.float32(nan)]
[np.float32(2.0), np.float32(3.0), np.float32(4.0), np.float32(9.0), np.float32(nan)]
[np.float32(2.0), np.float32(4.0), np.float32(9.0), np.float32(nan)]


In [3]:
statVars = []
for timeWindow in range(20,130,10):
    statVarList= [f'mean WSPD {timeWindow}min',
             f'std. dev. WSPD {timeWindow}min',
             f'mean WSPD_10N {timeWindow}min',
             f'std. dev. WSPD_10N {timeWindow}min',
             f'mean Ux_10N {timeWindow}min',
             f'std. dev. Ux_10N {timeWindow}min',
             f'mean Vy_10N {timeWindow}min',
             f'std. dev. Vy_10N {timeWindow}min',
             f'mean WDIR {timeWindow}min',
             f'std. dev. WDIR {timeWindow}min',
             f'mean cosWDIR {timeWindow}min',
             f'std. dev. cosWDIR {timeWindow}min',
             f'mean sinWDIR {timeWindow}min',
             f'std. dev. sinWDIR {timeWindow}min']
    statVars = statVars + statVarList

In [4]:
def getSubDF(ds, LAT, LON):
    subds = xr.Dataset()

    taoTimeVar = ['TAO_TIME']
    
    windVars = ['WDIR',
         'WDIR_QC',
         'WDIR_DM',
         'WSPD',
         'WSPD_QC',
         'WSPD_DM',
         'UWND',
         'VWND']

    sstVars = ['SST',
             'SST_QC',
             'SST_DM']
    
    airtVars = ['AIRT',
             'AIRT_QC',
             'AIRT_DM']
    
    rhVars = ['RELH',
             'RELH_QC',
             'RELH_DM']
    
    u10Vars =['WSPD_10N',
             'WSPD_10']

    statVars = []
    for timeWindow in range(20,130,10):
        statVarList= [f'mean WSPD {timeWindow}min',
                 f'std. dev. WSPD {timeWindow}min',
                 f'mean WSPD_10N {timeWindow}min',
                 f'std. dev. WSPD_10N {timeWindow}min',
                 f'mean Ux_10N {timeWindow}min',
                 f'std. dev. Ux_10N {timeWindow}min',
                 f'mean Vy_10N {timeWindow}min',
                 f'std. dev. Vy_10N {timeWindow}min',
                 f'mean WDIR {timeWindow}min',
                 f'std. dev. WDIR {timeWindow}min',
                 f'mean cosWDIR {timeWindow}min',
                 f'std. dev. cosWDIR {timeWindow}min',
                 f'mean sinWDIR {timeWindow}min',
                 f'std. dev. sinWDIR {timeWindow}min']
        statVars = statVars + statVarList
        
    
    satVars = ['retrieved_wind_speed',
             'retrieved_wind_direction',
             'rain_impact',
             'flags',
             'eflags',
             'nudge_wind_speed',
             'nudge_wind_direction',
             'retrieved_wind_speed_uncorrected',
             'cross_track_wind_speed_bias',
             'atmospheric_speed_bias',
             'wind_obj',
             'ambiguity_obj',
             'number_in_fore',
             'number_in_aft',
             'number_out_fore',
             'number_out_aft',
             'gmf_sst',
             'distance_from_coast',
             'exp_bias_wrt_oceanward_neighbors']

    satAmbVars =['ambiguity_speed',
                 'ambiguity_direction']

     
    
    for windVar in windVars:
        subds[windVar] = ds[windVar].sel(HEIGHT=4)
    subds.drop_vars('HEIGHT')
    for sstVar in sstVars:
        subds[sstVar] = ds[sstVar].sel(DEPTH=1)
    subds.drop_vars('DEPTH')
    for rhVar in rhVars:
        subds[rhVar] = ds[rhVar].sel(HEIGHT=3)
    subds.drop_vars('HEIGHT')
    for airtVar in airtVars:
        subds[airtVar] = ds[airtVar].sel(HEIGHT=3)
    subds.drop_vars('HEIGHT')
    for u10Var in u10Vars:
        subds[u10Var] = ds[u10Var].sel(HEIGHT=10)
    subds.drop_vars('HEIGHT')

    for statVar in statVars:
        subds[statVar] = ds[statVar]

    subds['LATITUDE'] = xr.DataArray(LAT * np.ones(len(subds['QS_TIME']), dtype = float), dims=['QS_TIME'])
    subds['LONGITUDE'] = xr.DataArray(LON * np.ones(len(subds['QS_TIME']), dtype = float), dims=['QS_TIME'])
    
    subds['TAO_TIME'] = ds['TAO_TIME']

    deployFileName = f'../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_DeploymentDates.nc'
    ds2 = xr.open_dataset(deployFileName)
    startDates = np.array(ds2['startDate'])
    endDates = np.array(ds2['endDate'])

    depNum = np.zeros((len(subds['QS_TIME'])), dtype=int)
    c = 1
    for i in range(1,len(startDates)):
        mask = subds['QS_TIME'] >= startDates[i]
        mask *= subds['QS_TIME'] <= endDates[i]
        depNum[mask] = c
        c = c+1

    subds['Deployment index'] = xr.DataArray(depNum, dims=['QS_TIME'])
    
    
    satDS = ds[satVars]
    satDS = satDS.isel(cross_track=0)

    amb_speed = ds['ambiguity_speed'].isel(cross_track=0).to_numpy()
    num_amb_speed = np.sum(np.isnan(amb_speed), axis = 1)
    subds['num_ambiguities_speed'] = xr.DataArray(num_amb_speed, dims=['QS_TIME'])

    amb_direction = ds['ambiguity_direction'].isel(cross_track=0).to_numpy()
    num_amb_direction = np.sum(np.isnan(amb_direction), axis = 1)
    subds['num_ambiguities_direction'] = xr.DataArray(num_amb_direction, dims=['QS_TIME'])

    sat0DS = ds[satAmbVars]
    sat0DS = sat0DS.isel(cross_track=0, ambiguities=0)
    sat0DS = sat0DS.rename_vars({'ambiguity_speed': 'ambiguity_speed_0',
                        'ambiguity_direction': 'ambiguity_direction_0'})

    sat1DS = ds[satAmbVars]
    sat1DS = sat1DS.isel(cross_track=0, ambiguities=1)
    sat1DS = sat1DS.rename_vars({'ambiguity_speed': 'ambiguity_speed_1',
                        'ambiguity_direction': 'ambiguity_direction_1'})

    sat2DS = ds[satAmbVars]
    sat2DS = sat2DS.isel(cross_track=0, ambiguities=2)
    sat2DS = sat2DS.rename_vars({'ambiguity_speed': 'ambiguity_speed_2',
                        'ambiguity_direction': 'ambiguity_direction_2'})

    sat3DS = ds[satAmbVars]
    sat3DS = sat3DS.isel(cross_track=0, ambiguities=3)
    sat3DS = sat3DS.rename_vars({'ambiguity_speed': 'ambiguity_speed_3',
                        'ambiguity_direction': 'ambiguity_direction_3'})

    
    
    returnDS = xr.merge((subds, satDS, sat0DS, sat1DS, sat2DS, sat3DS))

    
    
    return returnDS

In [5]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)


allDS = xr.Dataset()
dataCount = 0
fileCount = 0
for task in taskList:
    lat = task[0]
    lon = task[1]

    LAT = lat
    LON = lon

    if lat < 0:
        latUnits = 'S'
    else:
        latUnits = 'N'

    if lon < 0:
        lonUnits = 'W'
    else:
        lonUnits = 'E'

    LON = (LON+360)%360
    lat=abs(lat)
    lon=abs(lon)

    matchFname = f'../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_MatchUp_720_mins_2000.nc'
    #deployFileName = f'../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_DeploymentDates.nc'
    
    if os.path.isfile(matchFname):
        print(f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_MatchUp_2000.nc')
        rds = xr.open_dataset(matchFname)
        ds = getSubDF(rds, LAT, LON)
        QS_TIME = ds['QS_TIME'].to_numpy()
        ds = ds.rename({'QS_TIME':'count'})
        dataLen = len(ds['count'])
        print(dataCount, dataCount+dataLen)
        ds['count'] = xr.DataArray(pd.Series(np.arange(dataCount, dataCount+dataLen)), 
                                   dims = ['count'])
        ds['QS_TIME'] = xr.DataArray(QS_TIME, 
                                   dims = ['count'])
        dataCount += dataLen
        if fileCount == 0:
            allDS = ds
        else:
            allDS = xr.concat((allDS, ds), dim='count')
        fileCount += 1
        

T_08S_095W_xrr_MatchUp_2000.nc
0 4091
T_08S_110W_xrr_MatchUp_2000.nc
4091 7839


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08S_125W_xrr_MatchUp_2000.nc
7839 12311


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08S_155W_xrr_MatchUp_2000.nc
12311 15908


...

/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08N_165E_xrr_MatchUp_2000.nc
233017 238343


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_09N_140W_xrr_MatchUp_2000.nc
238343 244387


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [6]:
allDS = allDS.drop_vars(['HEIGHT','DEPTH','cross_track','ambiguities'])

In [7]:
allDS

<xarray.Dataset>
Dimensions:                           (count: 244387, TAO_TIME_INDEX: 73,
                                       ambiguities: 4)
Coordinates:
  * count                             (count) int64 0 1 2 ... 244385 244386
  * TAO_TIME_INDEX                    (TAO_TIME_INDEX) int64 -36 -35 ... 35 36
Dimensions without coordinates: ambiguities
Data variables: (12/207)
    WDIR                              (count, TAO_TIME_INDEX) float32 nan ......
    WDIR_QC                           (count, TAO_TIME_INDEX) float32 nan ......
    WDIR_DM                           (count, TAO_TIME_INDEX) |S1 b'' ... b'D'
    WSPD                              (count, TAO_TIME_INDEX) float32 nan ......
    WSPD_QC                           (count, TAO_TIME_INDEX) float32 nan ......
    WSPD_DM                           (count, TAO_TIME_INDEX) |S1 b'' ... b'D'
    ...                                ...
    ambiguity_direction_1             (count) float64 143.9 141.1 ... 259.1
    ambiguity_speed_2                 (count) float64 nan nan ... 10.65 16.84
    ambiguity_direction_2             (count) float64 nan nan ... 253.4 340.0
    ambiguity_speed_3                 (count) float64 nan nan ... 9.297 16.84
    ambiguity_direction_3             (count) float64 nan nan ... 73.61 170.1
    QS_TIME                           (count) datetime64[ns] 2000-01-01T00:34...

In [8]:
extended_flag_meaning = ['rain_correction_not_applied_flag',
'correction_produced_negative_spd_flag',
'all_ambiguities_contribute_to_nudging_flag',
'large_rain_correction_flag',
'coastal_processing_applied_flag',
'lake_winds_flag',
'rain_nearby_flag',
'ice_nearby_flag',
'significant_rain_correction_flag',
'rain_correction_applied_flag',
'wind_retrieval_possibly_corrupted_flag']

extended_flags = np.array(allDS['eflags'].to_numpy(), dtype=int)
eflags = np.array(['{:016b}'.format(val) for val in extended_flags])
rainCorrectionNotApplied = np.array([val[15] == '0' for val in eflags])
rainCorrectionApplied = np.array([val[4] == '1' for val in eflags])
significantRainCorrectionApplied = np.array([val[5] == '1' for val in eflags])
largeRainCorrectionApplied = np.array([val[12] == '1' for val in eflags])
rainMask = np.logical_or(rainCorrectionApplied, significantRainCorrectionApplied)
rainMask = np.logical_or(rainMask, largeRainCorrectionApplied)
rainMask = np.logical_or(rainMask, rainCorrectionNotApplied)

In [9]:
noRainMask = ~rainMask
mask1 = allDS.sel(TAO_TIME_INDEX=0)['WSPD_QC'].isin([1,2]).to_numpy()
mask2 = allDS.sel(TAO_TIME_INDEX=0)['WDIR_QC'].isin([1,2]).to_numpy()
mask3 = allDS.sel(TAO_TIME_INDEX=0)['SST_QC'].isin([1,2]).to_numpy()
mask4 = allDS.sel(TAO_TIME_INDEX=0)['RELH_QC'].isin([1,2]).to_numpy()
mask5 = allDS.sel(TAO_TIME_INDEX=0)['AIRT_QC'].isin([1,2]).to_numpy()

In [10]:
selectMask = np.logical_and(noRainMask, mask1)
selectMask = np.logical_and(selectMask, mask2)
selectMask = np.logical_and(selectMask, mask3)
selectMask = np.logical_and(selectMask, mask4)
selectMask = np.logical_and(selectMask, mask5)

In [11]:
selectMask

array([ True,  True,  True, ...,  True,  True,  True])

In [12]:
indices = selectMask.nonzero()[0]

In [13]:
goodDataSet = allDS.sel(count=indices)

In [14]:
goodDataSet = goodDataSet.rename_vars({'WSPD':'Wind Speed (TAO)',
                  'WDIR':'Wind Direction (TAO)',
                  'UWND': 'Zonal Wind Speed (TAO)',
                  'VWND': 'Meridional Wind Speed (TAO)',
                  'SST': 'Sea Surface Temperature (TAO)',
                  'RELH': 'Relative Humidity (TAO)',
                  'AIRT': 'Air Temperature (TAO)',
                  'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                  'WSPD_10': 'Wind Speed at 10m (TAO)',
                  'retrieved_wind_speed': 'Wind Speed (QuikSCAT)',
                  'retrieved_wind_direction': 'Wind Direction (QuikSCAT)'})

In [15]:
goodDataSet.to_netcdf('goodData_QSwithTAO_withAmbiguities.nc')

In [16]:
centerData = allDS.sel(TAO_TIME_INDEX = 0)
centerData = centerData.drop('TAO_TIME_INDEX')
indices = list(np.nonzero(~rainMask)[0])

goodData = centerData.isel(count=~rainMask)
goodData = goodData.isel(count = goodData["WSPD_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["WDIR_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["SST_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["RELH_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["AIRT_QC"].isin([1,2]))


print(len(goodData['count']))




156953


In [17]:
mask = np.logical_or(goodData['WSPD'] > 1000, goodData['WDIR'] > 3600)
#mask = np.logical_or(mask, abs(goodData['WSPD_10N'].to_numpy()) < 0.1)
mask = np.logical_or(mask, np.isnan(goodData['WSPD'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['WDIR'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['retrieved_wind_speed'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['retrieved_wind_direction'].to_numpy()))
print(np.sum(mask))
df = goodData.isel(count = ~mask)
#df = df.to_dataframe()

<xarray.DataArray 'WSPD' ()>
array(8226)


In [18]:
df

<xarray.Dataset>
Dimensions:                           (count: 148727, ambiguities: 4)
Coordinates:
  * count                             (count) int64 0 1 2 ... 244385 244386
Dimensions without coordinates: ambiguities
Data variables: (12/207)
    WDIR                              (count) float32 326.4 321.8 ... 232.4
    WDIR_QC                           (count) float32 2.0 2.0 2.0 ... 2.0 2.0
    WDIR_DM                           (count) |S1 b'D' b'D' b'D' ... b'D' b'D'
    WSPD                              (count) float32 5.39 5.3 ... 10.64 9.83
    WSPD_QC                           (count) float32 2.0 2.0 2.0 ... 2.0 2.0
    WSPD_DM                           (count) |S1 b'D' b'D' b'D' ... b'D' b'D'
    ...                                ...
    ambiguity_direction_1             (count) float64 143.9 141.1 ... 259.1
    ambiguity_speed_2                 (count) float64 nan nan ... 10.65 16.84
    ambiguity_direction_2             (count) float64 nan nan ... 253.4 340.0
    ambiguity_speed_3                 (count) float64 nan nan ... 9.297 16.84
    ambiguity_direction_3             (count) float64 nan nan ... 73.61 170.1
    QS_TIME                           (count) datetime64[ns] 2000-01-01T00:34...

In [19]:
df = df.rename_vars({'WSPD':'Wind Speed (TAO)',
                  'WDIR':'Wind Direction (TAO)',
                  'UWND': 'Zonal Wind Speed (TAO)',
                  'VWND': 'Meridional Wind Speed (TAO)',
                  'SST': 'Sea Surface Temperature (TAO)',
                  'RELH': 'Relative Humidity (TAO)',
                  'AIRT': 'Air Temperature (TAO)',
                  'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                  'WSPD_10': 'Wind Speed at 10m (TAO)',
                  'retrieved_wind_speed': 'Wind Speed (QuikSCAT)',
                  'retrieved_wind_direction': 'Wind Direction (QuikSCAT)'})

In [20]:
df['Speed Difference (QuikSCAT - TAO)'] = df['Wind Speed (QuikSCAT)'] - df['Neutral Wind Speed at 10m (TAO)']

## change true degrees to degrees where 0 deg mean east and 90 deg mean north 
df['Wind Direction (TAO)'] = (-(df['Wind Direction (TAO)'] - 90.0) + 360)%360
df['Wind Direction (QuikSCAT)'] = (-(df['Wind Direction (QuikSCAT)'] - 90.0) + 360)%360

## calculate zonal and meridional winds 
df['Zonal Neutral Wind Speed (TAO)'] = df['Neutral Wind Speed at 10m (TAO)'] * np.cos(np.deg2rad(df['Wind Direction (TAO)']))
df['Meridional Neutral Wind Speed (TAO)'] = df['Neutral Wind Speed at 10m (TAO)'] * np.sin(np.deg2rad(df['Wind Direction (TAO)']))

df['Zonal Wind Speed (QuikSCAT)'] = df['Wind Speed (QuikSCAT)'] * np.cos(np.deg2rad(df['Wind Direction (QuikSCAT)']))
df['Meridional Wind Speed (QuikSCAT)'] = df['Wind Speed (QuikSCAT)'] * np.sin(np.deg2rad(df['Wind Direction (QuikSCAT)']))

## direction diff in range (-180,180)
dirDiff = ((df['Wind Direction (QuikSCAT)'] - df['Wind Direction (TAO)'])+360)%360
dirDiff = xr.where(dirDiff > 180, dirDiff-360, dirDiff)
df['Direction Difference (QuikSCAT - TAO)'] = dirDiff

df['Zonal Wind Speed Difference (QuikSCAT - TAO)'] = df['Zonal Wind Speed (QuikSCAT)'] - df['Zonal Neutral Wind Speed (TAO)']
df['Meridional Wind Speed Difference (QuikSCAT - TAO)'] = df['Meridional Wind Speed (QuikSCAT)'] - df['Meridional Neutral Wind Speed (TAO)']

for timeWindow in range(20,130,10):
    df[f'Speed Difference (QuikSCAT - TAO {timeWindow} min mean)'] = df['Wind Speed (QuikSCAT)'] - df[f'mean WSPD_10N {timeWindow}min']

    ## change true degrees to degrees where 0 deg mean east and 90 deg mean north 
    df[f'mean WDIR {timeWindow}min'] = (-(df[f'mean WDIR {timeWindow}min'] - 90.0) + 360)%360

    ## direction diff in range (-180,180)
    dirDiff = ((df['Wind Direction (QuikSCAT)'] - df[f'mean WDIR {timeWindow}min'])+360)%360
    dirDiff = xr.where(dirDiff > 180, dirDiff-360, dirDiff)
    df[f'Direction Difference (QuikSCAT - TAO {timeWindow} min mean)'] = dirDiff

    ### 
    df[f'Zonal Wind Speed Difference (QuikSCAT - TAO {timeWindow} min mean)'] = df['Zonal Wind Speed (QuikSCAT)'] - df[f'mean Ux_10N {timeWindow}min']
    df[f'Meridional Wind Speed Difference (QuikSCAT - TAO {timeWindow} min mean)'] = df['Zonal Wind Speed (QuikSCAT)'] - df[f'mean Vy_10N {timeWindow}min']

In [21]:
df

<xarray.Dataset>
Dimensions:                                                         (
                                                                     count: 148727,
                                                                     ambiguities: 4)
Coordinates:
  * count                                                           (count) int64 ...
Dimensions without coordinates: ambiguities
Data variables: (12/259)
    Wind Direction (TAO)                                            (count) float32 ...
    WDIR_QC                                                         (count) float32 ...
    WDIR_DM                                                         (count) |S1 ...
    Wind Speed (TAO)                                                (count) float32 ...
    WSPD_QC                                                         (count) float32 ...
    WSPD_DM                                                         (count) |S1 ...
    ...                                                              ...
    Zonal Wind Speed Difference (QuikSCAT - TAO 110 min mean)       (count) float64 ...
    Meridional Wind Speed Difference (QuikSCAT - TAO 110 min mean)  (count) float64 ...
    Speed Difference (QuikSCAT - TAO 120 min mean)                  (count) float64 ...
    Direction Difference (QuikSCAT - TAO 120 min mean)              (count) float64 ...
    Zonal Wind Speed Difference (QuikSCAT - TAO 120 min mean)       (count) float64 ...
    Meridional Wind Speed Difference (QuikSCAT - TAO 120 min mean)  (count) float64 ...

In [22]:
df.to_netcdf('rainFlagRemovedBuoyDataBadQualityRemovedMatchupAmbuguitiesAdded.nc')